In [0]:
import numpy as np
with open('labels.txt','r') as f:
  labels = f.read()
with open('reviews.txt','r') as f:
  reviews = f.read()

In [4]:
print(reviews[:1000])
print()
print(labels[:20])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [0]:
from string import punctuation

#convert the entire review to lower case
reviews = reviews.lower()
#getting rid of punctuation
reviews = ''.join([c for c in reviews if c not in punctuation])

#split by new line and spaces
reviews_split = reviews.split('\n')
all_text = ''.join(reviews_split)

#create a list of words
words = all_text.split()

In [6]:
len(reviews_split)

25001

In [0]:
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints=[]
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

In [8]:
reviews_split[0]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t   '

In [10]:
len(vocab_to_int)

74072

In [0]:
# label conversion
labels_split = labels.split('\n')
encoded_labels = [1 if label == 'positive' else 0 for label in labels_split]


In [12]:
# Preprocess data to have fixed length
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [0]:
# remove any reviews with zero length from the reviews_ints list and their corresponding label in encoded_labels.


non_zero_idx = [ii for ii,review in enumerate(reviews_ints) if len(review) != 0]
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

In [0]:
# Padding our reviews to length of 200

def pad_features(reviews_ints, seq_length):
  features = np.zeros((len(reviews_ints), seq_length),dtype=int)
  for i,row in enumerate(reviews_ints):
    features[i, -len(row):] = np.array(row)[:seq_length]
  return features

In [15]:
seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

# print first 10 values of the first 30 batches 
print(features[:30,:10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

In [16]:
len(features)

25000

In [17]:
# Create Test, Train and Validation test
split_frac=0.8
split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]


test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


In [0]:
#Create Dataloaders and Batching
import torch
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

batch_size = 50

train_loader = DataLoader(train_data, shuffle = True, batch_size = batch_size)
valid_loader = DataLoader(valid_data, shuffle = True, batch_size = batch_size)
test_loader = DataLoader(test_data, shuffle = True, batch_size = batch_size)

In [19]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[   10,    43,   216,  ...,    15,  7861, 11853],
        [    0,     0,     0,  ...,  2791,     2,  1605],
        [    0,     0,     0,  ...,  1682, 15591,   359],
        ...,
        [    0,     0,     0,  ...,     4,  1792, 13417],
        [   54,    10,  1668,  ...,   291,    26,  1231],
        [ 1237,   681,  1702,  ...,    26,    91,   172]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
        0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0])


In [30]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [0]:
import torch.nn as nn

class SentimentRNN(nn.Module):

  def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, output_size, drop_prob=0.5):
    super(SentimentRNN, self).__init__()
    self.output_size = output_size
    self.n_layers = n_layers
    self.hidden_dim = hidden_dim
    
    # lstm & embedding layers
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim,hidden_dim,n_layers,dropout=drop_prob, batch_first=True)
    
    #dropout layer
    self.dropout = nn.Dropout(0.3)

    #linear layer
    self.fc = nn.Linear(hidden_dim, output_size)
    self.sig = nn.Sigmoid()
  
  def forward(self, x, hidden):

    # Perform a forward pass
    batch_size = x.size(0)

    x = x.long()
    embeds = self.embedding(x)
    lstm_out, hidden = self.lstm(embeds, hidden)
    #print("lstm_out shape ", lstm_out.shape)
    lstm_out = lstm_out.contiguous().view(-1,self.hidden_dim)
    #print("mod lstm_out shape ", lstm_out.shape)
    out = self.dropout(lstm_out)
    out = self.fc(out)
    #print("out shape ", out.shape)
    sig_out = self.sig(out)
    #print("sig_out shape ", sig_out.shape)
    sig_out = sig_out.view(batch_size, -1)
    sig_out = sig_out[:, -1]
    #print("final sig_out shape ", sig_out.shape)

    return sig_out, hidden
  
  def init_hidden(self, batch_size):
    ''' Initializes hidden state '''
    # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
    # initialized to zero, for hidden state and cell state of LSTM
    weight = next(self.parameters()).data
      
    if (train_on_gpu):
         hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
               weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
    else:
         hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
               weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
    return hidden

In [69]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, embedding_dim, hidden_dim, n_layers, output_size)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [0]:
# We'll also be using a new kind of cross entropy loss, which is designed to work with a single Sigmoid output. 
# BCELoss, or Binary Cross Entropy Loss, applies cross entropy loss to a single value between 0 and 1.

lr = 0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [71]:
epochs = 4

counter = 0
print_every = 100
clip = 5 # gradient clipping

if (train_on_gpu):
    net.cuda()
  
net.train()

for e in range(epochs):
  #initialize hidden state
  h = net.init_hidden(batch_size)

  for inputs, labels in train_loader:
    counter += 1
    if (train_on_gpu):
      inputs, labels = inputs.cuda(), labels.cuda()
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])
    print("hidden layer weights shape", len(h))

    # zero accumulated gradients
    net.zero_grad()

    # fetch output from the model
    output, h = net(inputs,h)

    # calculate loss and perform backprop
    loss = criterion(output.squeeze(), labels.float())
    loss.backward()

    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs
    nn.utils.clip_grad_norm(net.parameters(), clip)
    optimizer.step()

    # loss stats

    if counter % print_every == 0:
      # Get validation loss
      val_h = net.init_hidden(batch_size)
      val_losses = []
      net.eval()

      for inputs, labels in train_loader:
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
            val_h = tuple([each.data for each in h])

            if(train_on_gpu):
              inputs, labels = inputs.cuda(), labels.cuda()
            
            output, val_h = net(inputs, val_h)
            val_loss = criterion(output.squeeze(), labels.float())
            val_losses.append(val_loss.item())

      net.train()
      print("Epoch: {}/{}...".format(e+1, epochs),
            "Step: {}...".format(counter),
            "Loss: {:.6f}...".format(loss.item()),
            "Val Loss: {:.6f}".format(np.mean(val_losses)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch: 1/4... Step: 100... Loss: 0.705597... Val Loss: 0.669211
Epoch: 1/4... Step: 200... Loss: 0.719013... Val Loss: 0.639934
Epoch: 1/4... Step: 300... Loss: 0.525805... Val Loss: 0.553433
Epoch: 1/4... Step: 400... Loss: 0.605924... Val Loss: 0.603630
Epoch: 2/4... Step: 500... Loss: 0.496774... Val Loss: 0.452374
Epoch: 2/4... Step: 600... Loss: 0.497550... Val Loss: 0.389408
Epoch: 2/4... Step: 700... Loss: 0.311398... Val Loss: 0.388483
Epoch: 2/4... Step: 800... Loss: 0.374166... Val Loss: 0.348734
Epoch: 3/4... Step: 900... Loss: 0.242217... Val Loss: 0.300855
Epoch: 3/4... Step: 1000... Loss: 0.269293... Val Loss: 0.280373
Epoch: 3/4... Step: 1100... Loss: 0.278119... Val Loss: 0.236469
Epoch: 3/4... Step: 1200... Loss: 0.452162... Val Loss: 0.214456
Epoch: 4/4... Step: 1300... Loss: 0.318557... Val Loss: 0.177835
Epoch: 4/4... Step: 1400... Loss: 0.210594... Val Loss: 0.157533
Epoch: 4/4... Step: 1500... Loss: 0.094346... Val Loss: 0.153222
Epoch: 4/4... Step: 1600... Loss: 

In [72]:
  # Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))


Test loss: 0.512
Test accuracy: 0.799


In [0]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

In [0]:
def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")
        

In [0]:
# positive test review
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'

In [84]:
# call function
seq_length=200 # good to use the length that was trained on

predict(net, test_review_pos, seq_length)

Prediction value, pre-rounding: 0.987695
Positive review detected!
